In [ ]:
%reset

In [42]:
%load_ext autoreload
%autoreload 1
%aimport cooking_session
%run cooking_session.py

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
df_raw = pd.read_csv('files/ongridpilot_jan19.csv', sep=',')

In [7]:
min_size_of_spikes=0.0
df_spikes = df_raw.loc[(df_raw.energy > df_raw.energy.shift(-1) + min_size_of_spikes) &
                   (df_raw.meter_number == df_raw.meter_number.shift(-1))]
print('The data has this many spikes: ' + str(df_spikes.timestamp.count()))

The data has this many spikes: 0


In [25]:
# Parameters
time_resolution = 1

df_raw = resolve_spikes(df_raw)
df_processed = extract_cooking_events(df_raw, time_resolution=1, 
                             error_margin=0,
                             min_cooking_event=0.04,
                            event_min_current = 0,
                            min_active_load=0.05)

error_margin=0.01
df_epc = df_processed.copy()
df_epc.reset_index(inplace=True)

# Make on intermediate Cooking Event Count
df_epc['event_count'] = 0
df_epc.loc[(df_epc.cooking_event.diff()
                    != 0), 'event_count'] += 1
df_epc.event_count = df_epc['event_count'].cumsum()  

df_epc.loc[
(df_epc.cooking_event.isnull() == True),
'event_count'] = np.nan

# checking start of events
start_of_event = df_epc.copy()
start_of_event = start_of_event.groupby(
    ['meter_number', 'event_count']).head(1)
start_of_event.loc[
    ((start_of_event['energy'] -
      error_margin <= start_of_event['energy'].shift()) & (
        start_of_event.event_count.isnull() == False) & (
        start_of_event.meter_number == start_of_event.meter_number.shift())),
    'timestamp_issue'] = True

df_epc['timestamp_issue'] = df_epc.event_count.map(
    start_of_event.set_index('event_count')['timestamp_issue'].to_dict())

# checking end of events
end_of_event = df_epc.copy()
end_of_event = end_of_event.groupby(
    ['meter_number', 'event_count']).tail(1)
end_of_event.loc[
    ((end_of_event['energy'] -
      error_margin <= end_of_event['energy'].shift()) & (
        end_of_event.event_count.isnull() == False) & (
        end_of_event.meter_number == end_of_event.meter_number.shift())),
    'timestamp_issue'] = True

df_epc['timestamp_issue'] = df_epc.event_count.map(
    end_of_event.set_index('event_count')['timestamp_issue'].to_dict())

df_timestamp_issue = df_epc.loc[(df_epc['timestamp_issue'] == True)]
print('The data has this many timestamp issues: ' + str(df_timestamp_issue.timestamp.count()))

The data has this many timestamp issues: 0


In [26]:
df_epc = extract_cooking_events(df_raw, time_resolution=1, 
                             error_margin=0,
                             min_cooking_event=0.04,
                            event_min_current = 0,
                            min_active_load=0.05)
df_epc.reset_index(inplace=True)
df_epc.loc[
            (
                (df_epc.energy.shift(-1) - df_epc.energy > 0.0)
                & (df_epc.meter_number == df_epc.meter_number.shift(-1))
                  & (df_epc.cooking_event != df_epc.cooking_event.shift(-1))
            ), 'energy_gap_to_next'] = df_epc.energy.shift(-1) - df_epc.energy
df_epc.set_index('timestamp', inplace=True)
print('The data has this much missing energy consumption [kWh] before adding start and end of cooking events: ' + str(df_epc.energy_gap_to_next.sum()))

The data has this much missing energy consumption [kWh] before adding start and end of cooking events: 87.31999999999998


In [27]:
df_epc = preprocess_epc_data(df_raw, time_resolution=1, 
                             error_margin=0,
                             min_cooking_event=0.04,
                            event_min_current = 0,
                            min_active_load=0.05)
df_only_events = create_only_event_df(df_epc)
df_epc = clean_data_set(df_epc)

# Create a column of the energy gaps that still exist after cleaning the data
df_epc.reset_index(inplace=True)
df_epc.loc[
            (
                (df_epc.energy.shift(-1) - df_epc.energy > 0.0)
                & (df_epc.meter_number == df_epc.meter_number.shift(-1))
                  & (df_epc.event_count != df_epc.event_count.shift(-1))
            ), 'energy_gap_to_next'] = df_epc.energy.shift(-1) - df_epc.energy
df_epc.set_index('timestamp', inplace=True)
print('The data has this much missing energy consumption [kWh] after adding start and end of cooking events: ' + str(df_epc.energy_gap_to_next.sum()))

The data has this much missing energy consumption [kWh] after adding start and end of cooking events: 86.19333333333331


In [37]:
df_gaps = df_epc.copy()
df_gaps.reset_index(inplace=True)
#df_epc_meter = df_gaps.loc[df_gaps['meter_number']==86000174]
#df_gaps = df_epc_meter.copy()
df_gaps['energy_lost'] = df_gaps['energy_gap_to_next']
df_gaps.loc[(df_gaps['timestamp']-df_gaps['timestamp'].shift() > pd.to_timedelta(1, unit='m'))&\
        (df_gaps['meter_number'] == df_gaps['meter_number'].shift()), 'time_gap2'] = df_gaps['timestamp']-df_gaps['timestamp'].shift()
df_gaps['hour_gap'] = df_gaps['time_gap2']/np.timedelta64(1, 'h')
df_gaps = df_gaps.groupby(pd.cut(df_gaps['energy_gap_to_next'],
                    [0,0.01,0.02,0.05,0.07,0.085, 0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1000])).agg({'energy_lost':'sum', 'timezone_region':'count', 'hour_gap' : 'sum', 'meter_number' :'nunique'})
df_gaps.rename({'timezone_region':'count'},axis=1,inplace=True)
df_gaps

,energy_lost,count,hour_gap,meter_number
energy_gap_to_next,,,,
"(0.0, 0.01]",8.903333,909,529.433333,29
"(0.01, 0.02]",10.810000,940,1653.366667,29
"(0.02, 0.05]",6.483333,207,1953.800000,20
"(0.05, 0.07]",4.800000,82,925.283333,13
"(0.07, 0.085]",2.350000,32,180.666667,8
"(0.085, 0.1]",1.470000,16,80.416667,6
"(0.1, 0.2]",6.446667,46,155.500000,7
"(0.2, 0.3]",3.820000,16,88.300000,6
"(0.3, 0.4]",2.030000,6,0.250000,5


In [29]:
df_epc.reset_index(inplace=True)
df_epc.set_index('meter_number', inplace=True)
print(df_epc.energy_gap_to_next.nlargest(6))
df_epc.reset_index(inplace=True)
df_epc.set_index('timestamp', inplace=True)

meter_number
86000174    21.333333
86000174     3.280000
86000137     2.313333
86000142     2.030000
86000155     1.410000
86000134     1.200000
Name: energy_gap_to_next, dtype: float64


In [40]:
print('number of active meters in tz_cooking_pilot_2 region: ' + str(df_epc.energy_rank.nunique()))

number of active meters in tz_cooking_pilot_2 region: 33


In [30]:
energy_min = df_epc.groupby('meter_number').agg({'energy':'min'})
energy_max = df_epc.groupby('meter_number').agg({'energy':'max'})
energy_tot = energy_max - energy_min
print('energy [kWh] totally recorded by the SM: ' + str(energy_tot['energy'].sum()))

energy [kWh] totally recorded by the SM: 260.73


In [31]:
cooking_event_energy_min = df_epc.groupby('event_count').first()
cooking_event_energy_max = df_epc.groupby('event_count').last()
cooking_event_energy_tot = cooking_event_energy_max['energy'] - cooking_event_energy_min['energy']
print('energy [kWh] totally recorded in cooking events: ' + str(cooking_event_energy_tot.sum()))

energy [kWh] totally recorded in cooking events: 174.5366666666667


In [32]:
print('Total energy consumption [kWh]: ' + '{0:.2f}'.format(energy_tot['energy'].sum()))
print('Energy consumption [kWh], which is not part of a cooking session: ' + '{0:.2f}'.format(energy_tot['energy'].sum() - cooking_event_energy_tot.sum()))

Total energy consumption [kWh]: 260.73
Energy consumption [kWh], which is not part of a cooking session: 86.19


In [33]:
df_epc.event_count.nunique()

715

In [38]:
cooking_event_energy_min = df_epc.groupby('event_count').first()
cooking_event_energy_max = df_epc.groupby('event_count').last()
cooking_event_energy_tot = cooking_event_energy_max['energy'] - cooking_event_energy_min['energy']
cooking_event_energy_tot.sum()

174.5366666666667

In [39]:
energy_min = df_epc.groupby('meter_number').agg({'energy':'min'})
energy_max = df_epc.groupby('meter_number').agg({'energy':'max'})
energy_tot = energy_max - energy_min
energy_tot['energy'].sum()

260.73